# Kurtosis Demo

Notebook for playing around and demonstrating the spectral kurtosis estimator described on the wiki.

### Spectral Kurtosis Estimator

In [10]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
#%matplotlib qt

In [4]:
#create noise
#Should run this cell ONCE

#band should be a power of 2
band=1024
#m should be > ~30
m=6000
#n should be 1, but maybe doesnt integer divide m
n=1
#d must be 1 for AWGN
d=1

nsamp=band*m*n
noise_RMS=2**5

#temp lists for troubleshooting
#s1_temp=[]
#s2_temp=[]
#ratios=[]

#create noise
noise = np.random.normal(0.0,noise_RMS,nsamp)


data=noise#+sin

Now creating several sources of RFI. data=noise call in the beginning of each cell to make sure that we start with the already calculated noise values. Then, calculate and plot the SK estimator.

Run any interference-generating cell below to add it to the noise. You may want to comment out the 'data=noise' line if you are adding more than one interference source.

In [77]:
#data with one sin wave
data=noise
amp=10
per=5
data=noise+amp*np.sin(np.arange(nsamp)*2.0*math.pi/per)

In [60]:
#data with sin waves
data=noise
for per in range(10,30,2):
    data=data+4*np.sin(np.arange(nsamp)*2.0*math.pi/per)
#why did this cell get line numbers

In [73]:
#pulsed square wave

data=noise

height=100
dcycle=0.2
per=300

#start at sample 0, go 2000 pulses for total length per*amt samples
start=0
amt=2000

#create mask of 0's for adding pulse profile to
mask=np.zeros(nsamp)

#create series of pulses
for sampnum in range(start,start+per*amt,per):
    for i in range(0,int(per*dcycle)):
        mask[sampnum+i]=mask[sampnum+i]+height
data=noise+mask


In [75]:
#calculate sk

#DFT on band# of time samples, adds to ints
ints=[]
for intnum in range(m*n):
    temp=np.fft.fft(data[band*intnum:band*(intnum+1)])
    temp2=temp[band//2:]#drop first half
    ints.append(temp2.real**2+temp2.imag**2)
ints=np.array(ints)
#now ints contains m power spectra of arbitrary y-scale, of band/2 chans each

#calculate sk estimator for each channel
sk_est=[]
for chan in range(band//2):
    #make s1 and s2 as defined by whiteboard (by 2010b)
    #s2 definition will probably throw error if n does not integer divide m
    s1=sum(ints[:,chan])
    s2=sum(np.sum(ints[:,chan].reshape(-1,n)**2,axis=1))
    #-------------
    #s1_temp.append(s1)
    #s2_temp.append(s2)
    #ratios.append((m*s2)/(s1**2))
    #-------------
    #record sk estimator
    sk_est.append(((m*n*d+1)/(m-1))*((m*s2)/(s1**2)-1))
    
#sk_est now has the 'spectrum' of sk estimator values for each channel
#plot data result in r+, with noise result underneath in g+
plt.gcf().clear()
plt.plot(sk_est_noise,'g+')
plt.plot(sk_est,'r+')
plt.xlabel('Frequency Channel')
plt.ylabel('SK Estimator')
plt.show()

In [ ]:
#run this cell after evaluating SK estimator for noise. Uses result of cell directly above
sk_est_noise = sk_est

In [74]:
#plot time sample data
plt.gcf().clear()
#plt.plot(noise[1024:2048],'r+')
plt.plot(data[1024:2048],'b+')
plt.xlabel('Time (samples)')
plt.ylabel('Real Voltage')
plt.show()

In [ ]:
#plot histogram
plt.gcf().clear()
plt.hist(noise,fc=(0, 0, 1, 0.5),bins=range(-150,150,1))
plt.hist(sinone,fc=(1, 0, 0, 0.5),bins=range(-150,150,1))

## Computing Thresholds

Taken from Nick Joslyn's helpful code
https://github.com/NickJoslyn/helpful-BL/blob/master/helpful_BL_programs.py



In [7]:
print('M: '+str(m))
print('N: '+str(n))
print('d: '+str(d))

M: 6000
N: 1
d: 1


special.gammainc: Regularized Lower Incomplete Gamma Function
https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gammainc.html

optimize.newton: Find local 0 of a function
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.newton.html

I can't yet see how the equations in the paper translate to this, but this is replicated nearly exactly from IDL code given by the paper "EOVSA Implementation of a Spectral Kurtosis Correlator" https://arxiv.org/abs/1702.05391



In [11]:


def upperRoot(x, moment_2, moment_3, p):
    upper = np.abs( (1 - sp.special.gammainc( (4 * moment_2**3)/moment_3**2, (-(moment_3-2*moment_2**2)/moment_3 + x)/(moment_3/2/moment_2)))-p)
    return upper

def lowerRoot(x, moment_2, moment_3, p):
    lower = np.abs(sp.special.gammainc( (4 * moment_2**3)/moment_3**2, (-(moment_3-2*moment_2**2)/moment_3 + x)/(moment_3/2/moment_2))-p)
    return lower




def spectralKurtosis_thresholds(M, N = n, d = d, p = 0.0013499):

    Nd = N * d

    #Statistical moments
    moment_1 = 1
    moment_2 = ( 2*(M**2) * Nd * (1 + Nd) ) / ( (M - 1) * (6 + 5*M*Nd + (M**2)*(Nd**2)) )
    moment_3 = ( 8*(M**3)*Nd * (1 + Nd) * (-2 + Nd * (-5 + M * (4+Nd))) ) / ( ((M-1)**2) * (2+M*Nd) *(3+M*Nd)*(4+M*Nd)*(5+M*Nd))
    moment_4 = ( 12*(M**4)*Nd*(1+Nd)*(24+Nd*(48+84*Nd+M*(-32+Nd*(-245-93*Nd+M*(125+Nd*(68+M+(3+M)*Nd)))))) ) / ( ((M-1)**3)*(2+M*Nd)*(3+M*Nd)*(4+M*Nd)*(5+M*Nd)*(6+M*Nd)*(7+M*Nd) )
    #Pearson Type III Parameters
    delta = moment_1 - ( (2*(moment_2**2))/moment_3 )
    beta = 4 * ( (moment_2**3)/(moment_3**2) )
    alpha = moment_3 / (2 * moment_2)

    error_4 = np.abs( (100 * 3 * beta * (2+beta) * (alpha**4)) / (moment_4 - 1) )
    x = [1]
    upperThreshold = sp.optimize.newton(upperRoot, x[0], args = (moment_2, moment_3, p))
    lowerThreshold = sp.optimize.newton(lowerRoot, x[0], args = (moment_2, moment_3, p))
    return lowerThreshold, upperThreshold

In [12]:
spectralKurtosis_thresholds(300,N=10,d=1)

(0.76753581349215294, 1.2821180158638763)

## Load in real data instead of simulated

Taken from Richard's code

extractChan26.py is included in the github repo, but will not be used as that requires the GbtRaw module and downloading the 128 GB data file.

chan26.npy (100MB) contains the relevent data.

In [2]:
#!/users/rprestag/venv/bin/python

# from astropy.io import fits
import pylab
from matplotlib import rcParams

# from GbtRaw import *

def spectroFITS(array, tStart, tRes, fStart, fRes, file_name):
    """Writes out array as an image in a FITS file"""

    # create the dynamic spectrum as the primary image
    hdu = fits.PrimaryHDU(array)

    # add the axes information
    hdu.header['CRPIX1'] = 0.0
    hdu.header['CRVAL1'] = tStart
    hdu.header['CDELT1'] = tRes
    hdu.header['CRPIX2'] = 0.0
    hdu.header['CRVAL2'] = fStart
    hdu.header['CDELT2'] = fRes

    # create the bandpass and timeseries
    bandpass    = np.average(array, axis=1)
    timeseries  = np.average(array, axis=0)

    # and create new image extensions with these
    bphdu = fits.ImageHDU(bandpass,name='BANDPASS')
    tshdu = fits.ImageHDU(timeseries,name='TIMESERIES')
    # uodate these headers.
    bphdu.header['CRPIX1'] = 0.0
    bphdu.header['CRVAL1'] = fStart
    bphdu.header['CDELT1'] = fRes
    tshdu.header['CRPIX1'] = 0.0
    tshdu.header['CRVAL1'] = tStart
    tshdu.header['CDELT1'] = tRes


    hdulist = fits.HDUList([hdu, bphdu, tshdu])
    hdulist.writeto(file_name)

def main():

    rcParams.update({'figure.autolayout' : True})
    rcParams.update({'axes.formatter.useoffset' : False})

    pol = 0

    # create 512 time bins of 1024 spectra, each with 32 integrations.
    # there may be a few bytes not used.
    nfreq = 1024
    nint  = 32

    # get the data
    tsData = np.load("chan26.npy")
    tsLen = tsData.shape[0]

    # required polarization channels                                       
    sp = 2*pol
    ep = sp+2



    # empty list of power spectra
    spec_list = []

    print(tsLen)
    print(nfreq * nint)
    nspec = int(tsLen / (nfreq * nint))
    print("processing ", str(nspec), "spectra...")


    # do the work
    for s in range(nspec):
        print("spectrum: ", s)
        winStart = s * (nfreq * nint)
        accum = np.zeros(nfreq)
        for i in range(nint):
            start = winStart + i * nfreq
            end = start + nfreq
            in_arr = np.zeros((nfreq), dtype=np.complex_)
            in_arr.real = tsData[start:end, 0]
            in_arr.imag = tsData[start:end, 1]
            out_arr = np.fft.fftshift(np.fft.fft(in_arr))
            accum += np.abs(out_arr)**2
            spec_list.append(accum/nint)


    # convert back to numpy array and transpose to desired order
    dyn_spec = np.transpose(np.asarray(spec_list))

    # plot the results - first the spectrogram. Something is clearly wrong!
    pylab.imshow(dyn_spec)
    pylab.show()
    # This should be the time series - I am not sure why it looks like this...
    pylab.plot(np.average(dyn_spec, axis=0),"r+")
    pylab.show()
    # this is the average power-spectrum
    pylab.plot(np.average(dyn_spec, axis=1))
    pylab.show()
